<a href="https://colab.research.google.com/github/BrevinBrown/BrevinBrown/blob/main/ML_Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns # used for the confusion matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures


vars = ['year','manufacturer','model','condition','cylinders','size','type','paint_color','state']
# vars = ['region','year','manufacturer','model','condition','cylinders','size','type','paint_color','state','price']
dataset = pd.read_csv('/content/vehicles.csv' )

X = dataset[vars]
Y = dataset['price']

print(dataset.head())

           id                                                url  \
0  7222695916  https://prescott.craigslist.org/cto/d/prescott...   
1  7218891961  https://fayar.craigslist.org/ctd/d/bentonville...   
2  7221797935  https://keys.craigslist.org/cto/d/summerland-k...   
3  7222270760  https://worcester.craigslist.org/cto/d/west-br...   
4  7210384030  https://greensboro.craigslist.org/cto/d/trinit...   

                   region                         region_url  price  year  \
0                prescott    https://prescott.craigslist.org   6000   NaN   
1            fayetteville       https://fayar.craigslist.org  11900   NaN   
2            florida keys        https://keys.craigslist.org  21000   NaN   
3  worcester / central MA   https://worcester.craigslist.org   1500   NaN   
4              greensboro  https://greensboro.craigslist.org   4900   NaN   

  manufacturer model condition cylinders  ... size  type paint_color  \
0          NaN   NaN       NaN       NaN  ...  NaN   NaN

In [5]:
def cleanData(x,y):
    i = 0
    for column in x:
        drops = []
        #print(x.head(10))
        for var in x[column]:
            if pd.isna(var):
                #print(var)
                drops.append(i)
            i += 1
        y = y.drop(drops)
        x = x.drop(drops)
        x.reset_index(drop=True, inplace=True)
        y.reset_index(drop=True, inplace=True)
        i = 0  # reset i for the next column
    drops=[]
    i=0

    #drop crazy outlier values, someone put car in at over $3 billion
    for var in y:
      if (var>100000 or var <1000):
        drops.append(i)
      i+=1
    #print(drops)
    y = y.drop(drops)
    x = x.drop(drops)
    x.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)
    return x,y


In [6]:
def enumerateData(x):
    for column in x:
        strings=[]
        if (column != 'year'):
          for var in x[column]:
              if var not in strings:
                  strings.append(var)
                  index = len(strings) - 1
              else:
                  index = strings.index(var)
              x[column].replace(var, index, inplace=True)
              
        #print(strings)
    #print(x.head())
    return x


In [ ]:
xclean, Y = cleanData(X,Y)
print(len(Y))

In [ ]:
# Transforms the price into log space
#Y = np.log(Y)

In [ ]:
# Plots the listing prices of cars in the dataset after removing outliers (log scale)
plt.yscale("log")
plt.xlabel('Dataset Row')
plt.ylabel('Price Category ($)')
plt.title('Listing Prices')
plt.scatter(range(0,len(Y)),Y,2)
plt.show()

#x_y_enumerated = enumerateData(xclean.join(Y))
#sns.heatmap(abs(x_y_enumerated.corr()), annot=True, cmap='coolwarm')#, fmt=.2f) 
#plt.show()

In [ ]:
logbins = np.logspace(np.log10(1000),np.log10(100000),256)
plt.hist(Y, bins=logbins)
plt.xscale('log')
plt.xlabel('Listing Prices')
plt.ylabel('Number of Listings')
plt.title('Listing Price Histogram')
plt.show()

In [ ]:
#for var in vars:
# xclean, Y = cleanData(X,Y)

# instead of enumerating, this turns category columns into series of boolean columns for each category
xclean = pd.get_dummies(xclean, columns = ['manufacturer','model','condition','cylinders','size','type','paint_color','state'])
print(len(Y))
# xclean = enumerateData(xclean)

xclean.head()

In [ ]:
def MSE(y, y_pred):
  return (1/len(y)*(abs(y-y_pred))).sum()

In [ ]:
def Run_PCA(x, y):
  costMSE = []
  costtrain = []
  model = linear_model.LinearRegression() #linear model is bad fit
  #trying to do a poly fit
  #poly = PolynomialFeatures(degree=2, include_bias=False)
  #polyFeatures = poly.fit_transform(x.reshape(-1,1))

  i=3000
  #for i in range(7000,7000):
  #PCA
  pca = PCA(n_components=i)
  principleComponents = pca.fit_transform(x) 
  X_train, X_test, y_train, y_test = train_test_split(principleComponents, y, test_size=0.20, random_state=42)

  #Fit to poly regression
  #x_train_poly = poly.fit_transform(X_train)
  #x_test_poly = poly.fit_transform(X_test)


  reg = model.fit(X_train,y_train)
  Y_pred = reg.predict(X_test)
  Y_train_pred = reg.predict(X_train)
  
  
  '''
  #print(model.coef_)
  plt.plot(X_train[:,3],model.predict(x_train_poly),color='red',label='model')
  plt.scatter(X_train[:,3], y_train,label='Price Data')
  plt.xlabel('index')
  plt.ylabel('Price')
  plt.title('Linear Regression Fit')
  plt.legend()
  plt.show()
  '''
  costMSE.append(MSE(y_test,Y_pred))
  #costtrain.append(MSE(y_train, Y_train_pred))

  
  return costMSE#, costtrain

In [ ]:
def train():
  #errorMSE , errortrain= Run_PCA(xclean,Y)
  errorMSE = Run_PCA(xclean,Y)
  '''
  #plt.scatter(range(1,10),error, color = 'red', label='error')
  plt.plot(range(1,10),error)
  
  plt.xlabel('K value')
  plt.ylabel('error')
  plt.legend()
'''
  #plt.plot(7000, errortrain, 'o-',label = 'Training Data')
  plt.plot(3000, errorMSE, 'o-', color='red',label='Test Data')
  plt.xlabel('Number of principal components')
  plt.ylabel('Error')
  plt.title('PCA for linear regression')
  plt.legend()
  plt.show()




In [ ]:
train()